# 🗺️ #30DayMapChallenge 2024 - Day 2: Lines

## レーシング スルー トーキョー: Street Kart Adventure 🏎️

### 1. Understanding Lines in GIS: The Path to Adventure 🛣️

#### What are Line Features?

Lines are one of the three fundamental vector data types in GIS:
* They represent linear features with length but no area
* Perfect for mapping routes, roads, rivers, and transportation networks
* Consist of multiple connected points (vertices) forming a path
* Can store attributes like distance, speed limits, or traffic flow

#### Properties of Line Features
* **Direction**: Lines have a start and end point
* **Length**: Measurable distance along the path
* **Connectivity**: Can intersect with other lines to form networks
* **Attributes**: Can contain data about the route characteristics

### 2. Data Processing: Mapping Our カート Adventure 🗾

#### Data Sources
* **Route Data**: `.js` file containing key waypoints (チェックポイント)
* **Street Network**: Google Maps API road network
* **Points of Interest**: Notable landmarks (観光スポット)

#### Processing Steps

##### 1. **データ準備 (Data Preparation)**

In [1]:
# First, we install the required libraries
# folium is for creating interactive maps
# polyline is for encoding/decoding geographic coordinates
!pip install folium polyline -q

In [2]:
# Import necessary libraries
import json                     # For working with JSON data
import re                       # For working with regular expressions
import folium                  # For creating interactive maps
from folium.plugins import AntPath    # For creating animated paths on maps
from branca.element import Template, MacroElement  # For custom map elements
import requests                # For making HTTP requests
import pandas as pd           # For data manipulation
import polyline               # For working with geographic coordinates
from jinja2 import Template   # For creating HTML templates

In [3]:
# Create a directory to store our data
# The -p flag creates parent directories if they don't exist
!mkdir -p /content/data

# Download data file from GitHub
# -P specifies the directory where to save the file
!wget -P /content/data https://github.com/oechenique/30DayMapChallenge/raw/refs/heads/main/Notebooks/data/data_day2/DirectionsService.zip

# Unzip the downloaded file
# -d specifies the directory where to extract the files
!unzip /content/data/DirectionsService.zip -d /content/data

--2024-11-02 15:54:51--  https://github.com/oechenique/30DayMapChallenge/raw/refs/heads/main/Notebooks/data/data_day2/DirectionsService.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/oechenique/30DayMapChallenge/refs/heads/main/Notebooks/data/data_day2/DirectionsService.zip [following]
--2024-11-02 15:54:51--  https://raw.githubusercontent.com/oechenique/30DayMapChallenge/refs/heads/main/Notebooks/data/data_day2/DirectionsService.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10342 (10K) [application/zip]
Saving to: ‘/content/data/DirectionsService.zip’

DirectionsService.z 100%[======

In [4]:
# Read the file as text
# 'encoding="utf-8"' ensures proper handling of special characters
with open("/content/data/DirectionsService.js", "r", encoding="utf-8") as file:
    content = file.read()

# Use regex to extract the JSON part from the file
# The pattern r'\(\s*({.*})\s*\)' looks for JSON data between parentheses
# re.DOTALL allows the dot to match newlines
json_data = re.search(r'\(\s*({.*})\s*\)', content, re.DOTALL).group(1)

# Convert the JSON string into a Python dictionary
data = json.loads(json_data)

# Extract route information and create a list of dictionaries
# Each dictionary will contain details about a segment of the route
routes_data = []
for route in data["routes"]:                  # Loop through each route
    for leg in route["legs"]:                 # Loop through each leg (segment) of the route
        for step in leg["steps"]:             # Loop through each step in the leg
            routes_data.append({
                # Starting point address
                "start_address": leg["start_address"],
                # Destination address
                "end_address": leg["end_address"],
                # Distance in text format (e.g., "2.3 km")
                "distance_text": step["distance"]["text"],
                # Distance in meters
                "distance_value": step["distance"]["value"],
                # Duration in text format (e.g., "5 mins")
                "duration_text": step["duration"]["text"],
                # Duration in seconds
                "duration_value": step["duration"]["value"],
                # Starting point coordinates
                "start_lat": step["start_location"]["lat"],
                "start_lng": step["start_location"]["lng"],
                # Ending point coordinates
                "end_lat": step["end_location"]["lat"],
                "end_lng": step["end_location"]["lng"],
                # Navigation instructions
                "instructions": step["html_instructions"]
            })

# Convert the list of dictionaries into a pandas DataFrame
# This makes it easier to analyze and manipulate the data
df = pd.DataFrame(routes_data)

# Display the first 5 rows of the DataFrame
df.head()

,start_address,end_address,distance_text,distance_value,duration_text,duration_value,start_lat,start_lng,end_lat,end_lng,instructions
0,"2-chōme-17-13 Azumabashi, Sumida City, Tokyo 1...","6 Chome-5 Toyosu, Koto City, Tokyo 135-0061, J...",7 m,7,1 min,2,35.710077,139.802652,35.710012,139.802657,Head <b>south</b>
1,"2-chōme-17-13 Azumabashi, Sumida City, Tokyo 1...","6 Chome-5 Toyosu, Koto City, Tokyo 135-0061, J...",47 m,47,1 min,20,35.710012,139.802657,35.709985,139.802134,Turn <b>right</b> toward <b>墨堤通り</b>/<wbr/><b>...
2,"2-chōme-17-13 Azumabashi, Sumida City, Tokyo 1...","6 Chome-5 Toyosu, Koto City, Tokyo 135-0061, J...",0.2 km,159,1 min,60,35.709985,139.802134,35.708554,139.802242,Turn <b>left</b> onto <b>墨堤通り</b>/<wbr/><b>都道4...
3,"2-chōme-17-13 Azumabashi, Sumida City, Tokyo 1...","6 Chome-5 Toyosu, Koto City, Tokyo 135-0061, J...",0.4 km,357,1 min,77,35.708554,139.802242,35.707638,139.798573,Turn <b>right</b> onto <b>浅草通り</b>/<wbr/><b>清澄...
4,"2-chōme-17-13 Azumabashi, Sumida City, Tokyo 1...","6 Chome-5 Toyosu, Koto City, Tokyo 135-0061, J...",0.2 km,155,1 min,36,35.707638,139.798573,35.706346,139.798011,Slight <b>left</b> onto <b>清澄通り</b>/<wbr/><b>都...


##### 2. **ルート作成 (Route Creation)**

In [5]:
# Google Maps API key for accessing Google Maps services
google_api_key = "YOUR_API_KEY_HERE"

# Dictionary containing coordinates for important locations in Tokyo
# Each location is stored as a tuple of (latitude, longitude)
LOCATION_COORDINATES = {
    # Starting point - uses coordinates from the first row of our data
    'JAPANKART': (df.iloc[0]['start_lat'], df.iloc[0]['start_lng']),

    # Famous locations in Tokyo with their coordinates
    'Toyosu (Tokyo Bay)': (35.6412056, 139.7771503),
    'Ginza': (35.6718, 139.7640),              # Famous shopping district
    'Nihonbashi Bridge': (35.6832, 139.7716),  # Historic bridge
    'Akihabara Electric Town': (35.6998, 139.7734),  # Electronics/anime district
    'Ueno Station': (35.7106, 139.7760),       # Major railway station
    'Asakusa Sensoji Temple & Kaminarimon Gate': (35.7110864, 139.7963334),  # Famous temple
    'Tokyo Skytree': (35.7100, 139.8107),      # Tall broadcasting tower
}

# Map titles in Japanese and English
# Using emojis to make the title more visually appealing
japanese_title = "🏎️🚥 東京カートツアー: 東京都のスピードレース 🏁🏆 — 準備はいい？"
english_title = "Tokyo Kart Tour: Speed Racing through the Streets of Tokyo 🏁🏆"

##### 3. **見所追加 (Add Highlights)**

In [6]:
# Function to get route using Google Maps Directions API
def get_route(start, end):
    url = f"https://maps.googleapis.com/maps/api/directions/json?origin={start[0]},{start[1]}&destination={end[0]},{end[1]}&key={google_api_key}"
    response = requests.get(url)
    data = response.json()

    # Extract route points from the polyline
    route_points = []
    if data['status'] == 'OK':
        steps = data['routes'][0]['legs'][0]['steps']
        for step in steps:
            polyline_points = polyline.decode(step['polyline']['points'])
            route_points.extend(polyline_points)
    return route_points

# Create map with dark background (CartoDB dark_matter) without zoom controls
map_center = [df['start_lat'].mean(), df['start_lng'].mean()]
m = folium.Map(location=map_center, zoom_start=13, tiles='CartoDB dark_matter', control_scale=False, zoom_control=False)

# Add detailed routes to the map without intermediate points
for _, row in df.iterrows():
    start = (row['start_lat'], row['start_lng'])
    end = (row['end_lat'], row['end_lng'])

    # Get detailed route that follows streets
    route_points = get_route(start, end)

    # Draw animated route on the map
    AntPath(
        locations=route_points,
        color="#00FF00",  # Neon green color for Mario Kart effect
        weight=5,
        dash_array=[10, 20],
        pulse_color='white'  # White pulse effect
    ).add_to(m)

# Add notable points of interest
for location, coord in LOCATION_COORDINATES.items():
    folium.Marker(
        location=coord,
        popup=location,
        icon=folium.Icon(color='orange', icon='star')  # Distinctive icon for points of interest
    ).add_to(m)

# HTML for Japanese and English titles
title_html = f'''
<h3 align="center" style="font-size:20px">
    <b>{japanese_title}</b><br>
    <span style="font-size:16px">{english_title}</span>
</h3>
'''

# Create custom HTML for title and map
template_html = Template("""
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <title>{{ title }}</title>
    <style>
        body { background-color: black; color: white; font-family: sans-serif; }
    </style>
</head>
<body style="margin: 0;">
    <div style="padding: 10px; text-align: center;">
        {{ title_html | safe }}
    </div>
    {{ map_html | safe }}
</body>
</html>
""")

# Save the map file with Japanese and English titles
output_filename = "mario_kart_route_map.html"
with open(output_filename, "w") as f:
    f.write(template_html.render(title=japanese_title, title_html=title_html, map_html=m._repr_html_()))

print(f"Map generated as {output_filename} with dark background, Japanese and English titles, animated route, and points of interest!")

Map generated as mario_kart_route_map.html with dark background, Japanese and English titles, animated route, and points of interest!


In [7]:
# Display the generated interactive map in the notebook
m

### 3. Why Map Racing Routes? 🗾

Mapping go-kart routes helps:
* Visualize the most exciting parts of Tokyo
* Plan the best photo opportunities
* Understand traffic patterns
* Identify key landmarks along the way
* Share the experience with future drivers

### 4. Best Practices for Route Mapping 🎯

#### Design Considerations
1. **Route Clarity**
   - Use distinct colors for different routes
   - Add directional arrows
   - Include start/end points
   - Mark key attractions

2. **Safety Information**
   - Traffic zones
   - Speed limits
   - Pedestrian areas
   - Rest stops

3. **Points of Interest**
   - Photo spots
   - Cultural landmarks
   - Gaming references
   - Rest areas


### 5. Map Design Elements 🎨

#### Visual Hierarchy
* Primary routes in bold colors
* Secondary paths in lighter shades
* Landmarks as distinctive markers
* Clear labels for key locations

#### Theme Integration
* Gaming-inspired symbols
* Retro-style map elements
* Power-up station markers
* Character costume change points


### 6. Safety and Regulations 🚦

Important considerations:
* Follow all traffic rules
* Maintain safe speeds
* Stay with the guide
* Respect local regulations
* Wear provided safety gear
* No throwing bananas or shells! 🍌

### 7. Share Your Adventure! 📸

Tips for documenting your journey:
* Create photo stops at key points
* Record GPS tracks
* Share on social media with #StreetKartTokyo
* Contribute to the community map

### 8. Additional Resources 📚
* [Tokyo Street Kart Official Website](https://www.streetkart.jp/)
* [Tokyo Tourism Guide](https://www.gotokyo.org/)
* [Japanese Traffic Rules](https://www.jaf.or.jp/e/road.htm)

### 9. Safety First! (安全第一！)
* ヘルメット必須 (Helmet required)
* ガイドの指示に従う (Follow guide's instructions)
* 交通ルールを守る (Obey traffic rules)
* 青甲羅投げ禁止！(No blue shells!)

### さようなら (Goodbye) Until Tomorrow! 🚀
明日また会いましょう (See you tomorrow) for the next mapping challenge!

Remember: リアルマリオカート is all about safe fun! 🌟

In [8]:
#@title Follow Me!
%%html

<div style="background: linear-gradient(90deg, #1a1a1a, #2d2d2d); color: white; padding: 2rem; border-radius: 10px; font-family: 'Roboto', sans-serif; margin-top: 2rem;">
    <h2 style="text-align: center; margin-bottom: 1rem;">
        <span style="display: block; color: white; font-size: 2.7rem; margin-bottom: 0.5rem;">ありがとうございます！</span>
        <span style="display: block; color: white; font-size: 1.8rem;">🌍 Thank you for making it this far! 🎌</span>
    </h2>

    <p style="font-size: 1.2rem; margin-bottom: 1.5rem;">If you're interested in the world of geoinformatics and want to keep learning, connect with me! | 一緒に学びましょう！</p>

    <div style="display: flex; flex-wrap: wrap; gap: 1rem; margin-bottom: 1rem;">
        <a href="https://x.com/GastonEchenique" target="_blank" style="text-decoration: none; color: white; display: flex; align-items: center; background: #1DA1F2; padding: 0.5rem 1rem; border-radius: 5px; transition: transform 0.2s;">
            <svg style="width: 24px; height: 24px; margin-right: 0.5rem;" viewBox="0 0 24 24" fill="currentColor">
                <path d="M18.244 2.25h3.308l-7.227 8.26 8.502 11.24H16.17l-5.214-6.817L4.99 21.75H1.68l7.73-8.835L1.254 2.25H8.08l4.713 6.231zm-1.161 17.52h1.833L7.084 4.126H5.117z"/>
            </svg>
            <span>@GastonEchenique</span>
        </a>

        <a href="https://www.linkedin.com/in/gaston-echenique/" target="_blank" style="text-decoration: none; color: white; display: flex; align-items: center; background: #0A66C2; padding: 0.5rem 1rem; border-radius: 5px; transition: transform 0.2s;">
            <svg style="width: 24px; height: 24px; margin-right: 0.5rem;" viewBox="0 0 24 24" fill="currentColor">
                <path d="M20.447 20.452h-3.554v-5.569c0-1.328-.027-3.037-1.852-3.037-1.853 0-2.136 1.445-2.136 2.939v5.667H9.351V9h3.414v1.561h.046c.477-.9 1.637-1.85 3.37-1.85 3.601 0 4.267 2.37 4.267 5.455v6.286zM5.337 7.433c-1.144 0-2.063-.926-2.063-2.065 0-1.138.92-2.063 2.063-2.063 1.14 0 2.064.925 2.064 2.063 0 1.139-.925 2.065-2.064 2.065zm1.782 13.019H3.555V9h3.564v11.452zM22.225 0H1.771C.792 0 0 .774 0 1.729v20.542C0 23.227.792 24 1.771 24h20.451C23.2 24 24 23.227 24 22.271V1.729C24 .774 23.2 0 22.222 0h.003z"/>
            </svg>
            <span>Gastón Echenique</span>
        </a>

        <a href="https://github.com/oechenique" target="_blank" style="text-decoration: none; color: white; display: flex; align-items: center; background: #333; padding: 0.5rem 1rem; border-radius: 5px; transition: transform 0.2s;">
            <svg style="width: 24px; height: 24px; margin-right: 0.5rem;" viewBox="0 0 24 24" fill="currentColor">
                <path d="M12 0c-6.626 0-12 5.373-12 12 0 5.302 3.438 9.8 8.207 11.387.599.111.793-.261.793-.577v-2.234c-3.338.726-4.033-1.416-4.033-1.416-.546-1.387-1.333-1.756-1.333-1.756-1.089-.745.083-.729.083-.729 1.205.084 1.839 1.237 1.839 1.237 1.07 1.834 2.807 1.304 3.492.997.107-.775.418-1.305.762-1.604-2.665-.305-5.467-1.334-5.467-5.931 0-1.311.469-2.381 1.236-3.221-.124-.303-.535-1.524.117-3.176 0 0 1.008-.322 3.301 1.23.957-.266 1.983-.399 3.003-.404 1.02.005 2.047.138 3.006.404 2.291-1.552 3.297-1.23 3.297-1.23.653 1.653.242 2.874.118 3.176.77.84 1.235 1.911 1.235 3.221 0 4.609-2.807 5.624-5.479 5.921.43.372.823 1.102.823 2.222v3.293c0 .319.192.694.801.576 4.765-1.589 8.199-6.086 8.199-11.386 0-6.627-5.373-12-12-12z"/>
            </svg>
            <span>oechenique</span>
        </a>

        <a href="https://oechenique.github.io/geoanalytics/" target="_blank" style="text-decoration: none; color: white; display: flex; align-items: center; background: #2ecc71; padding: 0.5rem 1rem; border-radius: 5px; transition: transform 0.2s;">
            <svg style="width: 24px; height: 24px; margin-right: 0.5rem;" viewBox="0 0 24 24" fill="currentColor">
                <path d="M12 0c-6.627 0-12 5.373-12 12s5.373 12 12 12 12-5.373 12-12-5.373-12-12-12zm1 16.057v-3.057h2.994c-.059 1.143-.212 2.24-.456 3.279-.823-.12-1.674-.188-2.538-.222zm1.957 2.162c-.499 1.33-1.159 2.497-1.957 3.456v-3.62c.666.028 1.319.081 1.957.164zm-1.957-7.219v-3.015c.868-.034 1.721-.103 2.548-.224.238 1.027.389 2.111.446 3.239h-2.994zm0-5.014v-3.661c.806.969 1.471 2.15 1.971 3.496-.642.084-1.3.137-1.971.165zm2.703-3.267c1.237.496 2.354 1.228 3.29 2.146-.642.234-1.311.442-2.019.607-.344-.992-.775-1.91-1.271-2.753zm-7.241 13.56c-.244-1.039-.398-2.136-.456-3.279h2.994v3.057c-.865.034-1.714.102-2.538.222zm2.538 1.776v3.62c-.798-.959-1.458-2.126-1.957-3.456.638-.083 1.291-.136 1.957-.164zm-2.994-7.055c.057-1.128.207-2.212.446-3.239.827.121 1.68.19 2.548.224v3.015h-2.994zm1.024-5.179c.5-1.346 1.165-2.527 1.97-3.496v3.661c-.671-.028-1.329-.081-1.97-.165zm-2.005-.35c-.708-.165-1.377-.373-2.018-.607.937-.918 2.053-1.65 3.29-2.146-.496.844-.927 1.762-1.272 2.753zm-.549 1.918c-.264 1.151-.434 2.36-.492 3.611h-3.933c.165-1.658.739-3.197 1.617-4.518.88.361 1.816.67 2.808.907zm.009 9.262c-.988.236-1.92.542-2.797.9-.89-1.328-1.471-2.879-1.637-4.551h3.934c.058 1.265.231 2.488.5 3.651zm.553 1.917c.342.976.768 1.881 1.257 2.712-1.223-.49-2.326-1.211-3.256-2.115.636-.229 1.299-.435 1.999-.597zm9.924 0c.7.163 1.362.367 1.999.597-.931.903-2.034 1.625-3.257 2.116.489-.832.915-1.737 1.258-2.713zm.553-1.917c.27-1.163.442-2.386.501-3.651h3.934c-.167 1.672-.748 3.223-1.638 4.551-.877-.358-1.81-.664-2.797-.9zm.501-5.651c-.058-1.251-.229-2.46-.492-3.611.992-.237 1.929-.546 2.809-.907.877 1.321 1.451 2.86 1.616 4.518h-3.933z"/>
            </svg>
            <span>GeoAnalytics</span>
        </a>

        <a href="https://discord.com/users/gastonechenique" target="_blank" style="text-decoration: none; color: white; display: flex; align-items: center; background: #5865F2; padding: 0.5rem 1rem; border-radius: 5px; transition: transform 0.2s;">
            <svg style="width: 24px; height: 24px; margin-right: 0.5rem;" viewBox="0 0 24 24" fill="currentColor">
                <path d="M20.317 4.37a19.791 19.791 0 0 0-4.885-1.515a.074.074 0 0 0-.079.037c-.21.375-.444.864-.608 1.25a18.27 18.27 0 0 0-5.487 0a12.64 12.64 0 0 0-.617-1.25a.077.077 0 0 0-.079-.037A19.736 19.736 0 0 0 3.677 4.37a.07.07 0 0 0-.032.027C.533 9.046-.32 13.58.099 18.057a.082.082 0 0 0 .031.057a19.9 19.9 0 0 0 5.993 3.03a.078.078 0 0 0 .084-.028a14.09 14.09 0 0 0 1.226-1.994a.076.076 0 0 0-.041-.106a13.107 13.107 0 0 1-1.872-.892a.077.077 0 0 1-.008-.128a10.2 10.2 0 0 0 .372-.292a.074.074 0 0 1 .077-.01c3.928 1.793 8.18 1.793 12.062 0a.074.074 0 0 1 .078.01c.12.098.246.198.373.292a.077.077 0 0 1-.006.127a12.299 12.299 0 0 1-1.873.892a.077.077 0 0 0-.041.107c.36.698.772 1.362 1.225 1.993a.076.076 0 0 0 .084.028a19.839 19.839 0 0 0 6.002-3.03a.077.077 0 0 0 .032-.054c.5-5.177-.838-9.674-3.549-13.66a.061.061 0 0 0-.031-.03zM8.02 15.33c-1.183 0-2.157-1.085-2.157-2.419c0-1.333.956-2.419 2.157-2.419c1.21 0 2.176 1.096 2.157 2.42c0 1.333-.956 2.418-2.157 2.418zm7.975 0c-1.183 0-2.157-1.085-2.157-2.419c0-1.333.955-2.419 2.157-2.419c1.21 0 2.176 1.096 2.157 2.42c0 1.333-.946 2.418-2.157 2.418z"/>
            </svg>
            <span>Gaston | ガストン</span>
        </a>
    </div>

    <p style="font-size: 1rem; color: #888;">💡 頑張りましょう！Don't miss more content about GIS, Data Science and Geoinformatics!</p>

    <div style="margin-top: 1rem; padding-top: 1rem; border-top: 1px solid #444; font-size: 0.9rem; color: #666;">
        Made with 🗺️ & 💻 by Gaston Echenique | エチェニケ・ガストン
    </div>
</div>